<h3> creamos un objeto puerto que contiene el nombre, el calado maximo y el peso de la carga del puerto

In [2]:
import sys

class puerto:
    def __init__(self, puerto, calmax, peso):
        self.id = puerto
        self.calmax = calmax 
        self.peso = peso

<h3>funcion que sirve para calcular el calado del barco si asumimos que el peso principal del barco son 10 toneladas

In [3]:
def calcular_calado(carga_extra):
    peso_actual = 10 + carga_extra # Peso del barco en toneladas + el peso extra de los puertos
    densidad_agua = 1000  # Densidad del agua en kilogramos por metro cúbico
    peso_barco_kilos = peso_actual * 1000  # Convertir toneladas a kilogramos

    # Calcula el calado utilizando la fórmula de Arquímedes: calado = peso / (densidad * gravedad)
    calado = peso_barco_kilos / (densidad_agua * 9.8)  # Se asume una gravedad de 9.8 m/s^2
    return round(calado,2)

<h3>guardamos la lista de distancia dentro de la funcion pero también se podria haber dado de manera externa y calculamos la distancia

In [4]:
def calcular_distancia(puerto1, puerto2, distancias):
    return distancias.get((puerto1, puerto2), float('inf'))

<h3>Funcion que utiliza backtracking con poda para poder llegar a la solción del problema

In [5]:
#usamos una funcion dentro de otra funcion para no usar tanto codigo en el main y poder retornar las variables finales como el peso final, la distancia y la ruta
def viajero_ambulante_backtracking(puertos, distancias):
    n = len(puertos)
    visitado = [False] * n
    ruta_actual = [0] * (n + 1)
    mejor_ruta = [0] * (n + 1)
    mejor_distancia = sys.maxsize
    peso_final = 0
    
    #funcion que realiza el backtracking con poda
    def backtrack(pos, distancia_actual, peso_extra):
        nonlocal mejor_distancia
        nonlocal peso_final

        if pos == n:
            distancia = distancia_actual + calcular_distancia(puertos[ruta_actual[pos - 1]].id, puertos[0].id, distancias)
            if distancia < mejor_distancia:
                mejor_distancia = distancia 
                mejor_ruta[:] = ruta_actual[:]
                peso_final = peso_extra
            return

        for i in range(n):
            #Restriccion que dice si un puerto no fue visitado y si el peso que tiene el puerto + el peso que ya esta dentro del barco no excede el calado maximo del puerto
            if not visitado[i] and calcular_calado(peso_extra+puertos[i].peso) < puertos[i].calmax:
                nuevo_peso = peso_extra + puertos[i].peso
                visitado[i] = True
                ruta_actual[pos] = i
                nueva_distancia = distancia_actual + calcular_distancia(puertos[ruta_actual[pos - 1]].id, puertos[i].id, distancias)


                if nueva_distancia < mejor_distancia:
                    backtrack(pos + 1, nueva_distancia, nuevo_peso)

                visitado[i] = False

    visitado[0] = True
    backtrack(1, 0, 0)

    return [puertos[i].id for i in mejor_ruta], mejor_distancia, peso_final

<h3>Utilizamos el codigo

In [6]:
import time

In [16]:
# Ejemplo de uso
puertos = []
distancias = {}
characters = 65

nombre = input("Escriba el nombre del archivo(sin .txt): ")
nombre = nombre + ".txt"

with open(nombre, mode="r") as archivo:
    for linea in archivo:
        datos = linea.split()
        if (len(datos) == 2):
            puertos.append(puerto(chr(characters), float(datos[0]), float(datos[1])))
            characters += 1
        elif len(datos) == 3:
            distancias[(datos[0], datos[1])] = float(datos[2])

start = time.time()
mejor_ruta, mejor_distancia, peso = viajero_ambulante_backtracking(puertos, distancias)
end = time.time()

if mejor_distancia == sys.maxsize:
    print("No se puede recorrer todos los puertos debido a que 1 o más puertos se excede en su calado máximo")
else:
    print("Mejor ruta:", mejor_ruta)
    print("Mejor distancia:", mejor_distancia)
    print("El peso:", peso)
    print("Se demoro",end - start, "Segundos") 

Mejor ruta: ['A', 'C', 'D', 'B', 'A']
Mejor distancia: 80.0
El peso: 8.0
Se demoro 0.0 Segundos
